In [1]:
import numpy as np
import pandas as pd


def check_vals_in_arr(data, ref):
    cond = [value in ref for value in data]
    return cond

def approx_cords(df,key,approx):
    return [str(round(cor,approx)) for cor in df[key]]

def notArray(arr):
    return ~ np.array(arr)

def removeSampleValues(df,sample):
    condiX = check_vals_in_arr(df['Coordenada X'],sample['Coordenada X'].values)
    condiY = check_vals_in_arr(df['Coordenada Y'],sample['Coordenada Y'].values)
    condiDir = check_vals_in_arr(df['Unnamed: 11'],sample['Unnamed: 11'].values)
    condi = notArray(condiX and condiY and condiDir)
    return df[condi]
    

## Caculation of the sample size

In [39]:
sdv = 0.36 # kg/dia
mean = 0.79 # kg/dia
N = 1886
Z = 1.96
E = 0.08 * mean
(Z**2 * N * sdv**2)/((N-1)* E**2 + (Z**2 * sdv**2))


116.97817059179923

## **Dataset processing**
This dataset is not available publicly

In [2]:
df = pd.read_excel(r'db_carmen_raw.xlsx',sheet_name='Energía')

### Which usages are available in the dataset?

In [3]:
usage = df['Estrato'].drop_duplicates()
usage

0        Alumbrado Público
1                  excento
5                       11
1442                     1
2471                     2
11059                    3
20432                    4
20866                    5
20984                    6
21034                   12
21236                   13
21323          Provisional
Name: Estrato, dtype: object

## Urban Sample
In the next cell, the dataframe is filtered to choose only the values which correspond to the urban zone.

In [4]:
urban_sample_size = [2,50,79,2,2,0]
urban_df = df[df['Zona'] == 'Urbana']

### Sample extraction
The next cell uses pandas module to extract the sample acording to the desired sample size (*__change for your particular sizing__*), and export to excel.

In [86]:
keys = urban_df.keys()
final_sample_df = pd.DataFrame(columns=keys)
for stratum in range(1,7):
    stratified_df = urban_df[urban_df['Estrato'] == stratum]
    sample_size = urban_sample_size[stratum-1]
    sample = stratified_df.sample(n=sample_size)
    final_sample_df = pd.concat([final_sample_df,sample])
# final_sample_df.to_excel(r'Muestra urbana Carmen de Viboral.xlsx',index=False)


/var/folders/86/v681v5wn503_g_j110hv5slm0000gn/T/ipykernel_2569/3963063119.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_sample_df = pd.concat([final_sample_df,sample])


In [103]:
df = pd.read_excel(r'Muestra urbana Carmen de Viboral.xlsx')

## Non Residential Sample
In the next cell, the dataframe is filtered to choose only the values which correspond to the uses *Industrial, Comercial* y *Oficial*.

In [139]:
non_residential_sample_size = 140

# Conditions to filter the dataframe
a = df['Uso'] == 'Industrial'
b = df['Uso'] == 'Comercial'
c = df['Uso'] == 'Oficial'

nr_df = df[a|b|c]
nr_df

nr_sample = nr_df.sample(n=non_residential_sample_size,replace=False)
# nr_sample.to_excel(r'Muestra no residencial Carmen.xlsx',index=False)

# Fixing spatially redundant data (Non Residential Sample)
Removing data which corresponds to the exactly same coordinates, and the ones that presented a bad Urban, Rural clasification

In [143]:
'''Filtering the data that corresponds to the codes required to be changed'''

nr_sample_df = pd.read_excel(r'Muestra no residencial Carmen.xlsx')
codes_to_replace = pd.read_excel(r"nr_codes_to_change.xlsx")
data = nr_sample_df['codeid']
ref = codes_to_replace['CODES'].values
# condition = [value in ref for value in data]
# nr_sample_df[condition]

### Filtering the raw dataframe for non residential, urban individuals and cleaning it by removing alues with the same coordinates

In [144]:
approx = 8
# Conditions to filter the dataframe
a = df['Uso'] == 'Industrial'
b = df['Uso'] == 'Comercial'
c = df['Uso'] == 'Oficial'
d = df['Zona'] == 'Urbana'
e = df['Tipo Direccion'] == 'Urbana'

nr_df = df[(a|b|c) & d & e]
nr_df['Coordenada X'] = approx_cords(nr_df, 'Coordenada X', approx)
nr_df['Coordenada Y'] = approx_cords(nr_df, 'Coordenada Y', approx)
nr_df = nr_df.drop_duplicates(subset=['Coordenada X','Coordenada Y'])


nr_sample_df['Coordenada X'] = approx_cords(nr_sample_df, 'Coordenada X', approx)
nr_sample_df['Coordenada Y'] = approx_cords(nr_sample_df, 'Coordenada Y', approx)

# nr_sample_df = nr_sample_df.drop_duplicates(subset=['Coordenada X','Coordenada Y'])



/var/folders/86/v681v5wn503_g_j110hv5slm0000gn/T/ipykernel_2725/640396358.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nr_df['Coordenada X'] = approx_cords(nr_df, 'Coordenada X', approx)
/var/folders/86/v681v5wn503_g_j110hv5slm0000gn/T/ipykernel_2725/640396358.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nr_df['Coordenada Y'] = approx_cords(nr_df, 'Coordenada Y', approx)


### Filtering the data that is not currently present in the sample (via unique coordinates)

In [172]:
condiX = check_vals_in_arr(nr_df['Coordenada X'],nr_sample_df['Coordenada X'].values)
condiY = check_vals_in_arr(nr_df['Coordenada Y'],nr_sample_df['Coordenada Y'].values)
condiDir = check_vals_in_arr(nr_df['Unnamed: 11'],nr_sample_df['Unnamed: 11'].values)
condi = ~np.array(condiX or condiY or condiDir)
nr_df = nr_df[condi]

### Selection of a subsample to replace the redundant data

In [173]:
new_sample = nr_df.sample(n=1, replace=False)
new_sample['Coordenada X'] = [float(value) for value in new_sample['Coordenada X']]
new_sample['Coordenada Y'] = [float(value) for value in new_sample['Coordenada Y']]
# new_sample.to_excel(r'datos_nr_extra.xlsx',index=False)


# _Fixing spatially redundant data (Urban Residential Sample)_
Removing data which corresponds to the exactly same coordinates, and the ones that presented a bad Urban, Rural clasification

In [206]:
'''Filtering the data that corresponds to the codes required to be changed'''

urban_sample_df = pd.read_excel(r'Muestra urbana Carmen de Viboral.xlsx')
# urban_sample_df = urban_df.head(10)
codes_to_replace = ['UR1001','UR2001']
data = urban_sample_df['codeid']
# ref = codes_to_replace['CODES'].values
condition = notArray(check_vals_in_arr(data,codes_to_replace))
# nr_sample_df[condition]

## Descomentar linea 8 (urban_df = df[a & b & c])

In [181]:
approx = 8
# Conditions to filter the dataframe
a = df['Uso'] == 'Residencial'
b = df['Zona'] == 'Urbana'
c = df['Tipo Direccion'] == 'Urbana'

# Filtering the full urban dataset and converting coordinates to str
urban_df = df[a & b & c]
urban_df['Coordenada X'] = approx_cords(urban_df, 'Coordenada X', approx)
urban_df['Coordenada Y'] = approx_cords(urban_df, 'Coordenada Y', approx)
urban_df = urban_df.drop_duplicates(subset=['Coordenada X', 'Coordenada Y'])

# Converting coordinates to str
urban_sample_df['Coordenada X'] = approx_cords(urban_sample_df, 'Coordenada X', approx)
urban_sample_df['Coordenada Y'] = approx_cords(urban_sample_df, 'Coordenada Y', approx)
output_df = removeSampleValues(urban_df,urban_sample_df)

/var/folders/86/v681v5wn503_g_j110hv5slm0000gn/T/ipykernel_2725/4085557606.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  urban_df['Coordenada X'] = approx_cords(urban_df, 'Coordenada X', approx)
/var/folders/86/v681v5wn503_g_j110hv5slm0000gn/T/ipykernel_2725/4085557606.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  urban_df['Coordenada Y'] = approx_cords(urban_df, 'Coordenada Y', approx)


In [204]:
keys = output_df.keys()
subsample = pd.DataFrame(columns=keys)
reselected_sizes = [12,28,1,1]
for stratum in range(2,6):
    stratified_df = output_df[output_df['Estrato'] == stratum]
    sample_size = reselected_sizes[stratum-2]
    sample = stratified_df.sample(n=sample_size)
    subsample = pd.concat([subsample,sample])

In [205]:
subsample.to_excel(r'rechosen_urb.xlsx',index=False)

In [201]:
test_sample = urban_sample_df.drop_duplicates(subset=['Coordenada X', 'Coordenada Y'])
codes_to_replace = ['UR2003','UR2004','UR2014','UR2019','UR2023','UR2030','UR3005','UR3025','UR3032','UR3046','UR3075','UR5001','UR5002']
duplicates = notArray(check_vals_in_arr(urban_sample_df['codeid'],test_sample['codeid'].values))
reviewed = np.array(check_vals_in_arr(urban_sample_df['codeid'],codes_to_replace))
duplicates = urban_sample_df[reviewed|duplicates]

In [207]:
# Converting coordinates to str
urban_sample_df['Coordenada X'] = approx_cords(urban_sample_df, 'Coordenada X', approx)
urban_sample_df['Coordenada Y'] = approx_cords(urban_sample_df, 'Coordenada Y', approx)

In [208]:
urban_s_df_no_dups = urban_sample_df.drop_duplicates(subset=['Coordenada X', 'Coordenada Y'])

# Subsample for densities

In [25]:
codes_df = pd.read_excel('total_codes_sampling.xlsx')
rural_density_sample = pd.DataFrame(data = codes_df.values[110:138]).sample(12)
rural_density_sample.to_excel('density_rural_nr.xlsx',index=False)

In [17]:
urban_vals = codes_df.drop([i for i in range(110,139)])
urban_sample = urban_vals.sample(136)

# __Rural Sample__

In [2]:
rural_df = pd.read_excel(r'rural_raw.xlsx')
rural_df = rural_df[rural_df['Uso'] == 'Residencial']
rural_df = rural_df.drop(columns=['FID'])
rural_df = rural_df.rename(columns={'Coordenada':'Coordenada X','Coordena_1':'Coordenada Y'})

### Filtering the data to ensure that no spatially redundant data is present in the dataset

In [3]:
approx = 8
rural_df['Coordenada X'] = approx_cords(rural_df, 'Coordenada X', approx)
rural_df['Coordenada Y'] = approx_cords(rural_df, 'Coordenada Y', approx)
rural_df = rural_df.drop_duplicates(subset=['Coordenada X', 'Coordenada Y'])

### Choosing the sample using the filtered data

In [6]:
keys = ['codeid'] + [i for i in rural_df.keys()] 
rural_sample = pd.DataFrame(columns=keys)
sizes = [10,73,26,6,2,2]
for stratum in range(1,7):
    codeid = []
    stratified_df = rural_df[rural_df['Estrato'] == stratum]
    sample_size = sizes[stratum-1]
    sample = stratified_df.sample(n=sample_size)
    for cod in range(sample_size):
        codeid.append('RR'+str(stratum*1000 + cod + 1))
    sample.insert(1,'codeid',codeid,False)
    rural_sample = pd.concat([rural_sample,sample])
rural_sample['Coordenada X'] =  [float(i) for i in rural_sample['Coordenada X'].values]
rural_sample['Coordenada Y'] =  [float(i) for i in rural_sample['Coordenada Y'].values]

In [7]:
# rural_sample.to_excel(r'rural_sample.xlsx',index=False)